# Transactions Feature Engineering

In [19]:
import random

import pandas as pd

In [20]:
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path('../data/')
unzipped_data_path = data_path / 'unzipped'

# Unzip the raw customers data
with zipfile.ZipFile(data_path / 'raw' / 'transactions_train.csv.zip', 'r') as zip_ref:
    print('Unzipping transactions dataset...')
    zip_ref.extractall(unzipped_data_path)

Unzipping transactions dataset...


### Data Inspection

First, we inspect the transactions data

In [21]:
transactions_df = pd.read_csv(unzipped_data_path / 'transactions_train.csv')
transactions_df.shape

(31788324, 5)

In [22]:
transactions_df.head(5)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [23]:
transactions_df.iloc[random.randint(0, len(transactions_df))]

t_dat                                                      2020-02-29
customer_id         bc34bd1673bf2fbf66171e838a755d25251e6dde6de96f...
article_id                                                  804732006
price                                                        0.042356
sales_channel_id                                                    2
Name: 22876351, dtype: object

In [24]:
transactions_df.isnull().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [25]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


Seems like the dataset is pretty clean already. We only need to:
- Convert the `article_id` into string
- Convert `t_dat` to epoch milliseconds.
- Extract year, month, day, and day of week from `t_dat`

In [ ]:
transactions_df = transactions_df.assign(
    article_id=lambda df: df['article_id'].astype(str),  # Convert article_id to string
    t_dat=lambda df: pd.to_datetime(df['t_dat']),  # Convert t_dat to datetime
)

# Extract year, month, day, and day of week from t_dat
transactions_df = transactions_df.assign(
    year=lambda df: df['t_dat'].dt.year,
    month=lambda df: df['t_dat'].dt.month,
    day=lambda df: df['t_dat'].dt.day,
    day_of_week=lambda df: df['t_dat'].dt.weekday,  # Monday=0, Sunday=6
)

# Turn t_dat into epoch milliseconds
transactions_df = transactions_df.assign(
    t_dat=lambda df: df['t_dat'].astype(int) // 10**6
)

In [27]:
transactions_df.head(5)

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
0,1537401600000,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2018,9,20,3
1,1537401600000,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,2018,9,20,3
2,1537401600000,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,2018,9,20,3
3,1537401600000,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,2018,9,20,3
4,1537401600000,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,2018,9,20,3


### Save the Engineered Features
Lastly, we pickle it to save the processed transactions data

In [28]:
transactions_df.to_pickle(data_path / 'processed' / 'transactions.pkl')